In [30]:
library(BSgenome.Dmelanogaster.UCSC.dm3)
library(motifmatchr) # (https://bioconductor.org/packages/release/bioc/html/motifmatchr.html)
library(Biostrings)
library(SummarizedExperiment)
library(Matrix)


Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand




In [2]:
# using PWM log-odds
load("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_12_20/TF_clusters_PWMs.RData")

In [35]:
# Set working directory
setwd("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq")

In [3]:
# load in the training fasta
fasta <- readDNAStringSet("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq/analysis/2023_11_26/seqdatasets/Sequences_Train.fa")
fasta

DNAStringSet object of length 402296:
         width seq                                          names               
     [1]   249 ATTCAGATTGCCTCTCATTGT...TGCCTCTCATTCACTCTCTT chr2L_5587_5835_+...
     [2]   249 AAATGGCCGCTCAAGAAAAGG...ATATATTTAAAATGCCGATC chr2L_5778_6026_+...
     [3]   249 ATAAGGATCAAAAAGTCCTGA...TCTACAGATTCTTCTAAGTT chr2L_14226_14474...
     [4]   249 TTTCCATGACTGACTGGAATG...ATTGTGAACCACATTTTCAA chr2L_18618_18866...
     [5]   249 TCTATCGACCCATAGCCGTAG...TTTGATGCATGTGTAGTGAG chr2L_34121_34369...
     ...   ... ...
[402292]   249 ATGCTTTGAGACAAAACATTG...AATATACAAATATACAAAAA chrYHet_310301_31...
[402293]   249 AGTTCCAATGGATTTTTGGGA...GTAACGTGCTCAATCTTAAC chrYHet_337801_33...
[402294]   249 TGCTGCTGCCGATATTGCTGT...CGCCAAAGGGACCGATACTG chrYHet_341501_34...
[402295]   249 AAACTAATCCATGCTTCTTTC...GATTGACTGCGCATGGCTAC chrYHet_341601_34...
[402296]   249 GGCTGTTGCTGTTCTCATGCT...GCCATCGTATTTCCGACGCC chrYHet_342501_34...

In [4]:
# Select 40,000 sequences at random from the training set
selected_fasta <- fasta[sample(length(fasta), 40000)]
selected_fasta

DNAStringSet object of length 40000:
        width seq                                           names               
    [1]   249 CGATAATTGTAAATCAAATTT...CGATTTATTCAGCTTTCTTAG chr2L_4993801_499...
    [2]   249 TTTTTCTGATATTATCATTGC...ATTAGACAGCCAGCATGTAAC chr3R_4714668_471...
    [3]   249 GCGTGTGTGTGTGTGTGTGTT...CCATCCTTCTCGCTCACATCC chrX_5573402_5573...
    [4]   249 AGTGCGCTCTCTCATTGCGCA...AATGTTTGCCTGAATTCCCAA chr2L_12461310_12...
    [5]   249 TTTGGGTTCAATAAGCCCCGG...GCAAGGCCCCATACTTTCTGA chr2L_13741449_13...
    ...   ... ...
[39996]   249 AAAAAAACAAGAAAGTGGGCG...ATACTCAAAGAAAGATCACCT chr3L_4007401_400...
[39997]   249 ATTGTTGGCAATACCCAAGGA...ATTTGCCAGCTGCGTTACCTG chr2L_155176_1554...
[39998]   249 AATAATATATGTATAGAGAAA...ATCGATATTTAGGAGCATACC chr3R_19033655_19...
[39999]   249 GGAGTGTGTATATGAACGCAA...TGATATCAATCAATGGCAATT chr3L_14647001_14...
[40000]   249 GCTGCCATGTGTGCGGATGTG...GTAGGGGCATGGGAAGGATTG chr2L_4464299_446...

In [41]:
motif_ix <- matchMotifs(
    TF_clusters_PWMs$All_pwms_log_odds,
    selected_fasta,
    genome = "BSgenome.Dmelanogaster.UCSC.dm3", 
    p.cutoff = 5e-4, 
    bg="even", 
    out = "scores"
)

In [42]:
# Grab motifCounts as df
motifCounts_df <- Matrix(motifCounts(motif_ix))
colnames(motifCounts_df) <- colData(motif_ix)$name
rownames(motifCounts_df) <- names(selected_fasta)
head(motifCounts_df)

  [[ suppressing 6502 column names ‘bergman__Abd-B’, ‘bergman__Adf1’, ‘bergman__Aef1’ ... ]]



6 x 6502 sparse Matrix of class "dgCMatrix"
                                                                               
chr2L_4993801_4994049_-_negative            . . 1 2 . . . 1 . . . . . 2 . 3 3 .
chr3R_4714668_4714916_-_peak_849bp_region   2 . . . . . . 1 . . 1 . 1 1 . 6 6 .
chrX_5573402_5573650_-_peak_849bp_region    1 . . . 1 . . . . . . . . 1 . 2 . .
chr2L_12461310_12461558_+_peak_849bp_region . . . 9 . . . . . . . 1 2 . . . 2 .
chr2L_13741449_13741697_-_peak_849bp_region 1 3 . 1 2 . 2 . . . . . . 1 . 2 1 .
chr2RHet_2774960_2775208_+_positive_peaks   . . . 1 1 . . . . . . 5 . . . 1 . .
                                                                               
chr2L_4993801_4994049_-_negative            . . 1 1 . . . . . . . . 1 1 1 . . .
chr3R_4714668_4714916_-_peak_849bp_region   . . 1 . 1 . 1 1 1 . . 1 . 1 1 . 1 .
chrX_5573402_5573650_-_peak_849bp_region    . . 1 . 2 . . . 1 . . . . 1 . . . .
chr2L_12461310_12461558_+_peak_849bp_region 2 . . . . . . . . . 1 1 2 . . . 

In [43]:
# Save as matrix market file
writeMM(motifScores_df, file = "training/2023_12_19/seqdatasets/motifCounts_df.mtx")
write.table(rownames(motifCounts_df), file = "training/2023_12_19/seqdatasets/motifCounts_df_row_names.txt", row.names = FALSE, col.names = FALSE)
write.table(colnames(motifCounts_df), file = "training/2023_12_19/seqdatasets/motifCounts_df_col_names.txt", row.names = FALSE, col.names = FALSE)

NULL

# DONE!

---